In [1]:
import os
import pandas as pd
import csv

def detect_delimiter(file_path):
    with open(file_path, 'r', newline='', encoding='utf-8') as file:
        dialect = csv.Sniffer().sniff(file.read(1024))
    return dialect.delimiter

delimiter = detect_delimiter('ERIHPLUSapprovedJournals.csv')
erih_plus_df = pd.read_csv('ERIHPLUSapprovedJournals.csv', sep=delimiter)

In [2]:
erih_plus_df.head(1)

,Journal ID,Print ISSN,Online ISSN,Original Title,International Title,Country of Publication,ERIH PLUS Disciplines,OECD Classifications,[Last Updated]
0,486254,1989-3477,NaN,@tic.revista d'innovació educativa,@tic.revista d'innovació educativa,Spain,Interdisciplinary research in the Social Scien...,Educational Sciences; Other Social Sciences,2015-06-25 13:48:26


In [3]:
def process_meta_csv(file_path, erih_plus_df):
    meta_data = pd.read_csv(file_path)
    meta_data['venue'] = meta_data['venue'].astype(str)
    meta_data['issn'] = meta_data['venue'].str.extract(r'issn:(\d{4}-\d{3}[\dX])')
    
    # Extract the identifier (OMID) from the 'id' column
    meta_data['id'] = meta_data['id'].str.extract(r'(meta:[^ ]*)')
    
    merged_data_print = erih_plus_df.merge(meta_data, left_on='Print ISSN', right_on='issn', how='inner')
    merged_data_online = erih_plus_df.merge(meta_data, left_on='Online ISSN', right_on='issn', how='inner')
    merged_data = pd.concat([merged_data_print, merged_data_online], ignore_index=True)
    
    # Keep only the relevant columns for the mapping dataframe
    merged_data = merged_data[['id', 'issn', 'Journal ID', 'Print ISSN', 'Online ISSN']].rename(columns={'id': 'OC_OMID', 'issn': 'OC_ISSN', 'Journal ID': 'EP_ID', 'Print ISSN': 'EP_Print_ISSN', 'Online ISSN': 'EP_Online_ISSN'})
    
    # Create the 'EP_ISSN' column
    merged_data['EP_ISSN'] = merged_data['EP_Print_ISSN'].combine_first(merged_data['EP_Online_ISSN'])
    
    # Drop the 'EP_Print_ISSN' and 'EP_Online_ISSN' columns
    merged_data = merged_data.drop(columns=['EP_Print_ISSN', 'EP_Online_ISSN'])

    return merged_data



In [4]:
csv_directory = 'I:\\open-sci\\dump-files\\opencitations-meta\\solo_one'
merged_data = pd.DataFrame()

for file_name in os.listdir(csv_directory):
    if file_name.endswith('.csv'):
        file_path = os.path.join(csv_directory, file_name)
        merged_data_file = process_meta_csv(file_path, erih_plus_df)
        merged_data = pd.concat([merged_data, merged_data_file], ignore_index=True)

In [5]:
#merged_data.to_csv('OpenCitations_Meta_ERIH_PLUS_mapping_3.csv', index=False)

In [6]:
merged_data.head(1)

,OC_OMID,OC_ISSN,EP_ID,EP_ISSN
0,meta:br/060100,NaN,488561,2341-0515


In [7]:
new_merged_data = merged_data.dropna(subset=['OC_ISSN']).reset_index(drop=True)
new_merged_data.head(2)

,OC_OMID,OC_ISSN,EP_ID,EP_ISSN
0,meta:br/0601646,0172-6404,471777,0172-6404
1,meta:br/0601638,0172-6404,471777,0172-6404


In [8]:
# Load DOAJ CSV file into a DataFrame
doaj_file_path = 'journalcsv__doaj.csv'  # Replace this with the actual file path
doaj_df = pd.read_csv(doaj_file_path, encoding="UTF-8")

In [11]:
doaj_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19278 entries, 0 to 19277
Data columns (total 54 columns):
 #   Column                                                                       Non-Null Count  Dtype  
---  ------                                                                       --------------  -----  
 0   Journal title                                                                19278 non-null  object 
 1   Journal URL                                                                  19278 non-null  object 
 2   URL in DOAJ                                                                  19278 non-null  object 
 3   When did the journal start to publish all content using an open license?     19277 non-null  float64
 4   Alternative title                                                            7485 non-null   object 
 5   Journal ISSN (print version)                                                 11148 non-null  object 
 6   Journal EISSN (online version)        

In [14]:
new_doaj = doaj_df.iloc[1:, [5, 6, 10]]
new_doaj.columns

Index(['Journal ISSN (print version)', 'Journal EISSN (online version)',
       'Country of publisher'],
      dtype='object')

In [16]:
# Merge the Open Access information with the merged_data DataFrame
merged_data_with_oa = new_merged_data.merge(new_doaj, left_on='EP_ISSN', right_on='Journal ISSN (print version)', how='left')
merged_data_with_oa = merged_data_with_oa.merge(new_doaj, left_on='EP_ISSN', right_on='Journal EISSN (online version)', how='left', suffixes=('_print', '_online'))

merged_data_with_oa.head()

# Save the resulting DataFrame to a new CSV file
#merged_data_with_oa.to_csv('OpenCitations_Meta_ERIH_PLUS_mapping_with_OA.csv', index=False)

,OC_OMID,OC_ISSN,EP_ID,EP_ISSN,Journal ISSN (print version)_print,Journal EISSN (online version)_print,Country of publisher_print,Journal ISSN (print version)_online,Journal EISSN (online version)_online,Country of publisher_online
0,meta:br/0601646,0172-6404,471777,0172-6404,NaN,NaN,NaN,NaN,NaN,NaN
1,meta:br/0601638,0172-6404,471777,0172-6404,NaN,NaN,NaN,NaN,NaN,NaN
2,meta:br/0601645,0172-6404,471777,0172-6404,NaN,NaN,NaN,NaN,NaN,NaN
3,meta:br/0601643,0172-6404,471777,0172-6404,NaN,NaN,NaN,NaN,NaN,NaN
4,meta:br/0601640,0172-6404,471777,0172-6404,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
merged_data_with_oa.columns

Index(['OC_OMID', 'OC_ISSN', 'EP_ID', 'EP_ISSN',
       'Journal ISSN (print version)_print',
       'Journal EISSN (online version)_print', 'Country of publisher_print',
       'Journal ISSN (print version)_online',
       'Journal EISSN (online version)_online', 'Country of publisher_online'],
      dtype='object')

In [18]:
# Drop rows with NaN values
merged_data_with_oa_no_nan = merged_data_with_oa.dropna(subset=['OC_OMID', 'OC_ISSN', 'EP_ID', 'EP_ISSN',
                                                                'Journal ISSN (print version)_print',
                                                                'Journal EISSN (online version)_print', 'Country of publisher_print',
                                                                'Journal ISSN (print version)_online',
                                                                'Journal EISSN (online version)_online', 'Country of publisher_online'])

merged_data_with_oa_no_nan.head()

,OC_OMID,OC_ISSN,EP_ID,EP_ISSN,Journal ISSN (print version)_print,Journal EISSN (online version)_print,Country of publisher_print,Journal ISSN (print version)_online,Journal EISSN (online version)_online,Country of publisher_online
